In [ ]:
# default_exp dataset_builder

# Dataset Builder
> Builds an optimal dataset with knowledge base relations, from a vanilla dataset.

In [ ]:
#export 
from rake_nltk import Rake
from kirby.database_proxy import WikiDatabase
import json

In [ ]:
#export
class DatasetBuilder():
    def __init__(self):
        self.rake = Rake()
        self.db = WikiDatabase()
        pass
    
    def build(self, ds, dataset_type='random'):
        "Build a database based a given dataset"
        if dataset_type == 'random':
            ds.map(self.random, batched=False)
        elif dataset_type == 'description':
            pass
        elif dataset_type == 'relevant':
            pass
        
    def keyword(self, x):
        ranked_phrases = self.get_ranked_phrases(x)
        return ranked_phrases[0]
    
    def random(self, x):
        keyword = self.keyword(x)
        e = self.db.get_entity_by_label(keyword)
        return e
    
    def get_ranked_phrases(self, x):
        self.rake.extract_keywords_from_text(x)
        return self.rake.get_ranked_phrases()
    
    def entity(self, ranked_phrases):
        "Queries the knowledge base to find the entity and it's relations"
        for phrase in ranked_phrases:
            entity = self.kba.get_entity(phrase)
            if entity is not None:
                return entity
        return entity         

# Testing

In [ ]:
# creation
ds_builder = DatasetBuilder()
assert isinstance(ds_builder, DatasetBuilder)

In [ ]:
# Test ranked phrases
x = "Stephen Curry is my favorite basketball player."
ds_builder.rake.extract_keywords_from_text(x)
ranked_phrases = ds_builder.rake.get_ranked_phrases()
ranked_phrases

['favorite basketball player', 'stephen curry']

In [ ]:
# Get random association
x = "Tim Cook from Apple announced a new iPhone"
random_association = ds_builder.random(x)
random_association

> <ipython-input-15-183c85c3fbb0>(24)random()
     22         keyword = self.keyword(x)
     23         import pdb; pdb.set_trace()
---> 24         e = self.db.get_entity_by_label(keyword)
     25         return e
     26 



ipdb>  s


--Call--
> /data/kirby/database_proxy.py(93)get_entity_by_label()
     91         return entities_id.values.tolist()
     92 
---> 93     def get_entity_by_label(self, label):
     94         """
     95         Search for the id, label and description of an specific identity



ipdb>  n


> /data/kirby/database_proxy.py(101)get_entity_by_label()
     99         :rtype: pandas.core.series.Series
    100         """
--> 101         entity_id = None
    102         table_name = self.get_table_name(label)
    103         try:



...

> /data/kirby/database_proxy.py(104)get_entity_by_label()
    102         table_name = self.get_table_name(label)
    103         try:
--> 104             entity_id = pd.read_sql_query(
    105                 "SELECT * FROM Entities_{} WHERE label = \"{}\";".format(table_name, label),
    106                 self.conn)



ipdb>  n


pandas.io.sql.DatabaseError: Execution failed on sql 'SELECT * FROM Entities_t WHERE label = "tim cook";': no such table: Entities_t
> /data/kirby/database_proxy.py(104)get_entity_by_label()
    102         table_name = self.get_table_name(label)
    103         try:
--> 104             entity_id = pd.read_sql_query(
    105                 "SELECT * FROM Entities_{} WHERE label = \"{}\";".format(table_name, label),
    106                 self.conn)



ipdb>  q


NameError: name 'exit' is not defined